## 1. 📚 Import Libraries

In [1]:
import itertools

import pandas as pd
import networkx as nx
import plotly.express as px
import plotly.graph_objects as go
from pyvis.network import Network

## 2. 📖 Read Dataframe

In [2]:
tracks_df = pd.read_csv("tracks.csv").drop(columns="Unnamed: 0")

tracks_df['artists'] = tracks_df['artists'].str.split(';')
tracks_df['artists'] = tracks_df['artists'].apply(lambda x: x if isinstance(x, list) else [])

tracks_df.head()

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,[Gen Hoshino],Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,[Ben Woodward],Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,"[Ingrid Michaelson, ZAYN]",To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,[Kina Grannis],Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,[Chord Overstreet],Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [3]:
artists_list = tracks_df['artists'].explode().unique().tolist()

multi_artist_tracks_df = tracks_df[tracks_df['artists'].apply(len) > 1]

uniquemulti_multi_artist_tracks_df = multi_artist_tracks_df['artists'].apply(lambda x: tuple(sorted(x))).unique()

print(f"""
Metric                         | Value
===============================|=======
All tracks                     | {len(tracks_df)}
-------------------------------|-------
All artists                    | {len(artists_list)}
-------------------------------|-------
Multi artist tracks            | {len(multi_artist_tracks_df)}
-------------------------------|-------
Multi artist percentage        | {round((len(multi_artist_tracks_df)/len(tracks_df))*100, 2)}%
-------------------------------|-------
Unique multi artist tracks     | {len(uniquemulti_multi_artist_tracks_df)}
-------------------------------|-------
Unique multi artist percentage | {round((len(uniquemulti_multi_artist_tracks_df)/len(tracks_df))*100, 2)}%
""")


Metric                         | Value
===============================|=======
All tracks                     | 114000
-------------------------------|-------
All artists                    | 29860
-------------------------------|-------
Multi artist tracks            | 30075
-------------------------------|-------
Multi artist percentage        | 26.38%
-------------------------------|-------
Unique multi artist tracks     | 16177
-------------------------------|-------
Unique multi artist percentage | 14.19%



## 5. 👨🏼‍🎤 Generate the artist Dataframe

In [4]:
def get_mode(x):
    if x.empty:
        return None
    modes = x.mode()
    if len(modes) > 1:
        return None
    return modes.iloc[0]

In [5]:
tracks_exploded_artists_df = tracks_df.explode('artists')

agg_dict = {
    'track_id': 'count',
    'popularity': 'mean',
    'duration_ms': 'mean',
    'danceability': 'mean',
    'energy': 'mean',
    'loudness': 'mean',
    'mode': get_mode,
    'speechiness': 'mean',
    'acousticness': 'mean',
    'instrumentalness': 'median',
    'liveness': 'median',
    'valence': 'median',
    'tempo': 'median',    
    'track_genre': get_mode
}

artists_df = tracks_exploded_artists_df.groupby('artists').agg(agg_dict).reset_index()

artists_df = artists_df.rename(columns={
    'artists': 'artist',
    'track_id': 'track_count'
})

artists_df.head()

,artist,track_count,popularity,duration_ms,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_genre
0,!nvite,2,23.000000,137367.500000,0.820500,0.519,-10.173000,NaN,0.270000,0.499000,0.011360,0.1285,0.415,84.9970,study
1,"""Cats"" 1983 Broadway Cast",3,26.666667,479648.666667,0.361333,0.411,-9.101333,1.0,0.075533,0.900333,0.052300,0.5220,0.164,96.3280,show-tunes
2,"""Little"" Jimmie Dickens",1,11.000000,156306.000000,0.479000,0.271,-10.347000,1.0,0.033400,0.841000,0.000000,0.5330,0.621,142.1760,honky-tonk
3,"""Pepo"" San Martín",1,21.000000,242028.000000,0.370000,0.467,-12.189000,0.0,0.085300,0.446000,0.000540,0.3240,0.339,133.2540,tango
4,"""Puppy Dog Pals"" Cast",2,57.500000,47966.500000,0.705000,0.856,-4.274000,NaN,0.124500,0.096000,0.000705,0.1475,0.915,138.0425,children


## 6. 🔎 Analyze Dataframes

In [6]:
def plot_heatmap(df, title=""):
    df_corr = df.copy()
    for col in df_corr.select_dtypes(include=['object', 'category']).columns:
        df_corr[col] = df_corr[col].astype('category').cat.codes

    corr_matrix = df_corr.corr()
    
    fig = go.Figure(data=go.Heatmap(
        z=corr_matrix.values,
        x=corr_matrix.columns,
        y=corr_matrix.index,
        text=corr_matrix.round(2).values,
        texttemplate="%{text}",
        colorscale='Viridis'
    ))

    fig.update_layout(
        title=f'Correlation Matrix {title}',
        showlegend=False
    )

    fig.show()

plot_heatmap(tracks_df.drop(columns="artists"), "Tracks")
plot_heatmap(artists_df, "Artists")

In [7]:
fig1 = px.scatter(
    artists_df,
    x='tempo',
    y='loudness',
    size='track_count',
    color='track_genre',
    symbol='mode',
    symbol_map={0: 'cross', 1: 'circle'}
)

fig1.update_layout(
    title='Insight',
    xaxis_title='Tempo',
    yaxis_title='Loudness',
    legend_title='Genre, Mode'
)


fig1.show()

## 7. 🕸️ Construct Graph  

In [8]:
G = nx.MultiGraph()

artists_df_indexed = artists_df.set_index('artist')

for artist_name, row in artists_df_indexed.iterrows():
    G.add_node(artist_name, **row.to_dict())

for _, row in tracks_df.iterrows():
    artists_in_track = row['artists']
    
    if len(artists_in_track) > 1:
        for source, target in itertools.combinations(artists_in_track, 2):
            if G.has_node(source) and G.has_node(target):
                edge_attrs = row.drop('artists').to_dict()
                G.add_edge(source, target, **edge_attrs)

print(f"Generated graph with {G.number_of_nodes()} artists (V) and {G.number_of_edges()} colaborations (E)")

Generated graph with 29859 artists (V) and 47569 colaborations (E)


## 8. 👁️‍🗨️ Visulize Graph

In [9]:
artists_df.columns

Index(['artist', 'track_count', 'popularity', 'duration_ms', 'danceability',
       'energy', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'track_genre'],
      dtype='object')

In [10]:
tracks_df.columns

Index(['track_id', 'artists', 'album_name', 'track_name', 'popularity',
       'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'time_signature', 'track_genre'],
      dtype='object')